In [1]:
# Activate logging
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [2]:
from getpass import getpass

openai_api_key = getpass("Enter OpenAI API key:")

In [3]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(
    similarity="cosine",
    index="document",
    embedding_field="embedding",
    embedding_dim=1536
)


/opt/homebrew/Caskroom/miniforge/base/envs/haystack-test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
from datasets import load_dataset

# load seven wonders dataset from datasets and write documents into document store

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
document_store.write_documents(dataset)


WARNING - datasets.builder -  Using custom data configuration bilgeyucel--seven-wonders-6536ac0e9c84233f
WARNING - datasets.builder -  Found cached dataset parquet (/Users/rjtannous/.cache/huggingface/datasets/bilgeyucel___parquet/bilgeyucel--seven-wonders-6536ac0e9c84233f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [4]:
from haystack.nodes import EmbeddingRetriever

# define retriever model , update dense vector embeddings on document store index and define generator model
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="text-embedding-ada-002",
    api_key=openai_api_key,
    max_seq_len=1024,
    top_k=4,
)
document_store.update_embeddings(retriever=retriever)

INFO - haystack.modeling.utils -  Using devices: CPU - Number of GPUs: 0
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model text-embedding-ada-002


In [5]:
# define the QA tool's prompt template

from haystack.nodes import PromptNode, PromptTemplate

QA_promptnode = PromptTemplate(
            name="zero-shot-QA", 
            prompt_text="You are a helpful and knowledgeable agent. Only Answer if the {documents} contain the answer. If the user question is not related to the provided {documents}, say I don't have an answer\n"
            "Question: {query}\n"
            "Answer:"
        )

In [6]:
# declare the QA pipeline's prompt node
QA_builder = PromptNode(model_name_or_path="text-davinci-003", api_key=openai_api_key, default_prompt_template=QA_promptnode)

In [7]:
# Build the QA pipeline tool
from haystack import Pipeline

pipe = Pipeline()
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
pipe.add_node(component=QA_builder, name="Generator", inputs=["Retriever"])

In [28]:
from haystack_memory.prompt_templates import memory_template
from haystack.agents import Agent, Tool
from haystack.nodes import PromptNode

# create agent prompt node and define our Agent "memory_agent"
prompt_node = PromptNode(model_name_or_path="text-davinci-003", api_key=openai_api_key, max_length=512,
                         stop_words=["Observation:"])
memory_agent = Agent(prompt_node=prompt_node, prompt_template=memory_template)
# Define the first tool: A document store QA tool based on the pipeline
search_tool = Tool(name="DocumentStore_QA",
                   pipeline_or_node=pipe,
                   description="Access this tool to find missing information needed to answer questions.",
                   output_variable="results")

In [29]:
from haystack_memory.memory import MemoryRecallNode

memory_database = []
memory_node = MemoryRecallNode(memory=memory_database)
memory_tool = Tool(name="Memory",
                   pipeline_or_node=memory_node,
                   description="Your memory. Always access this tool first to remember what you have learned.")

In [30]:
# Add the memory and the search tool to the agent
memory_agent.add_tool(search_tool)
memory_agent.add_tool(memory_tool)

In [31]:
from haystack_memory.utils import MemoryUtils

# Chat with the agent
memory_utils = MemoryUtils(memory_database=memory_database, agent=memory_agent)
result, conversation_memory = memory_utils.chat("where did zeus' golden sandals rest?")


Agent memory-shot-react started with {'query': "where did zeus' golden sandals rest?", 'params': None}
 understand what the question is asking.
Tool: Memory
Tool Input: Where did Zeus' golden sandals rest?
Observation: []
Thought:  This is not enough. Let's try another tool. 
Tool: DocumentStore_QA
Tool Input: Where did Zeus' golden sandals rest?



Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.77it/s]


Observation: Zeus' golden sandals rested upon a footstool decorated with an Amazonomachy in relief.
Thought:  This is the answer.
Final Answer: Zeus' golden sandals rested upon a footstool decorated with an Amazonomachy in relief.

In [32]:
print(conversation_memory)

["where did zeus' golden sandals rest?", "Zeus' golden sandals rested upon a footstool decorated with an Amazonomachy in relief."]


In [33]:
print(result)

{'query': "where did zeus' golden sandals rest?", 'answers': [<Answer {'answer': "Zeus' golden sandals rested upon a footstool decorated with an Amazonomachy in relief.", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {}}>], 'transcript': "You are a helpful agent. To achieve your goal of answering questions correctly, use step by step thinking and\nuse the following tools to look for answers: DocumentStore_QA: Access this tool to find missing information needed to answer questions.\nMemory: Your memory. Always access this tool first to remember what you have learned.. Give the tool the full question as input; the tools will respond with observations.\nRefrain from using prior internal knowledge or tools you don't have access to. You should avoid searching elsewhere and avoid using external sources.\nOnly use observations returned by the tools you have been given access to.\nUse the following f

In [34]:
result, conversation_memory = memory_utils.chat("which muslim general raided the city of Rhodes?")


Agent memory-shot-react started with {'query': 'which muslim general raided the city of Rhodes?', 'params': None}

Tool: Memory
Tool Input: which muslim general raided the city of Rhodes?
Observation: ["where did zeus' golden sandals rest?", "Zeus' golden sandals rested upon a footstool decorated with an Amazonomachy in relief."]
Thought:  This is not relevant to the question I am trying to answer.
Tool: DocumentStore_QA
Tool Input: which muslim general raided the city of Rhodes?



Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s]


Observation: Muawiyah I
Thought:  This is the general that I am looking for!
Final Answer: Muawiyah I